In [1]:
from elasticsearch import Elasticsearch
import requests
from  elasticsearch.helpers import scan as scan

es = Elasticsearch([{"host": "localhost", "port": 9200}])

In [2]:
import pandas as pd
from pandas.io.json import json_normalize
#df=json_normalize(docs)

In [65]:
body={
    "_source": [
        "context.user_id",
        "context.course_id",
        "referer",
        "time",
        "event.submission",
        "event.attempts",
        "event.problem_id" 
    ],
    "query": {
        "bool":{
            "must":[                
                {"match_phrase":{
                    "event_type": "problem_check"
                }},
                {"match":{
                    "event_source": "server"
                }}           
            ],
            "must_not":[
                {"match_phrase":{
                    "event_type": "problem_check_fail"
                }}
            ]            
        }
    }
}

def ret():
    for hit in scan(es,
         index="online_courses", 
         query=body
        ):        
        if hit.get("_source").get("event") :
            if hit.get("_source").get("event").get("submission")!={}:
                submission=str(hit.get("_source").get("event").get("submission")).split('correct\': ')[1:]
                results=[]
                for res in submission:
                    if res[0:5]=='False':
                        results.append(0)
                    elif res[0:4]=='True':
                        results.append(1)
                yield {                    
                    "user_id":hit.get("_source").get("context").get("user_id"),
                    "course_id":hit.get("_source").get("context").get("course_id"),
                    "time":hit.get("_source").get("time"),
                    "results":results,
                    "attempt_num":hit.get("_source").get("event").get("attempts"),
                    "theme_id":hit.get("_source").get("referer").split('courseware/')[-1].split('/')[1],
                    "problem_id":hit.get("_source").get("event").get("problem_id"),
        }
            
        
        


In [66]:
df=pd.DataFrame(ret()).sort_values(by=['time'])

In [67]:
df[:3]

,attempt_num,course_id,problem_id,results,theme_id,time,user_id
0,1,course-v1:spbu+SOCLAB+spring_2018,block-v1:spbu+SOCLAB+spring_2018+type@problem+...,"[0, 0, 1, 0, 0]",0edf9012bff546778914358cd1e48f4e,2018-02-15T09:05:34.388942+00:00,243054
1,1,course-v1:spbu+SOCLAB+spring_2018,block-v1:spbu+SOCLAB+spring_2018+type@problem+...,"[0, 0, 0, 0, 0]",0edf9012bff546778914358cd1e48f4e,2018-02-15T12:41:49.105627+00:00,492510
3,2,course-v1:spbu+SOCLAB+spring_2018,block-v1:spbu+SOCLAB+spring_2018+type@problem+...,"[0, 0, 0, 0, 0]",0edf9012bff546778914358cd1e48f4e,2018-02-15T12:45:45.648085+00:00,492510


In [98]:
idx = df.groupby(['course_id','theme_id','problem_id','user_id'])['attempt_num'].idxmax()

In [102]:
df=df.loc[idx, ['course_id','theme_id','problem_id','user_id','results']]

In [105]:
df.to_csv(path_or_buf='tests_results.csv')

In [133]:
df = pd.read_csv('tests_results.csv', sep=',', encoding='utf-8')

In [71]:
themes=df['theme_id'].unique()

In [72]:
users=df['user_id'].unique()

In [132]:
for theme in themes:
    themeInformation=df.loc[df['theme_id'] == theme]
    for user in users:
        userResultInTheme=themeInformation.loc[df['user_id'] == user]
        if userResultInTheme.shape[0]>1:
            userResultInTheme=userResultInTheme.groupby(['course_id','theme_id','user_id']).agg({'results':sum})
            userResultInTheme=userResultInTheme.reset_index()
        if userResultInTheme.shape[0]==1:
            #calculations
        #write doc in db dont forget about 0-size
        #think about user coefficients
        #maybe make user dictionary
    

---
0                        [0, 0, 1, 0, 0]
50              [1, 1, 1, 1, 1, 1, 1, 1]
51     [1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]
52                    [1, 1, 1, 0, 1, 1]
5                                 [1, 1]
156       [1, 1, 1, 1, 1, 0, 0, 1, 1, 0]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 243054   [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   243054   

                                             results  
0  [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
---
2                       [0, 0, 0, 0, 0]
6              [0, 0, 0, 0, 0, 0, 0, 0]
41    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
49                   [1, 1, 1, 0, 1, 1]
4                                [0, 0]
Name: results, dtype: ob

---
87             [1, 1, 1, 1, 1, 1, 1, 1]
78                      [1, 1, 1, 1, 1]
96             [1, 1, 1, 1, 1, 1, 1, 1]
82             [1, 1, 1, 1, 1, 1, 1, 1]
84    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
94                   [1, 1, 1, 0, 1, 1]
77                               [1, 1]
99       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
91                   [1, 1, 1, 1, 1, 1]
95                   [1, 1, 1, 1, 1, 1]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 559616   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   559616   

                                             results  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
---
171    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
179                            [

course-v1:spbu+SOCLAB+spring_2018 467046   [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   467046   

                                             results  
0  [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
---
187    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
61                    [0, 0, 0, 0, 0]
63           [1, 0, 1, 1, 1, 1, 1, 1]
75                 [1, 1, 1, 0, 1, 1]
62                             [1, 1]
112    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
158    [1, 1, 1, 1, 0, 1, 1, 1, 0, 0]
76                 [1, 1, 1, 1, 1, 1]
106                [1, 0, 1, 1, 1, 1]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 595666   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
                           course_id  u

0  [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...  
---
176    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
64                    [0, 0, 0, 0, 0]
25                             [0, 0]
120    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
123    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 259731   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   259731   

                                             results  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...  
---
31                    [0, 0, 0, 0, 0]
54           [1, 1, 1, 1, 1, 1, 1, 1]
37                             [1, 1]
104    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
114    [1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
Name: results, dtype: object
---*
 

Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 554852   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   554852   

                                             results  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...  
---
164    [1, 0, 0, 1, 0, 1, 1, 1, 1, 1]
180                            [1, 0]
24                             [1, 1]
113    [1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
124    [1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 444404   [1, 0, 0, 1, 0, 1, 1,

...

---
187    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
61                    [0, 0, 0, 0, 0]
63           [1, 0, 1, 1, 1, 1, 1, 1]
75                 [1, 1, 1, 0, 1, 1]
62                             [1, 1]
112    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
158    [1, 1, 1, 1, 0, 1, 1, 1, 0, 0]
76                 [1, 1, 1, 1, 1, 1]
106                [1, 0, 1, 1, 1, 1]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 595666   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   595666   

                                             results  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...  
---
73              [1, 1, 1, 1, 1, 1, 1, 1]
169       [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
68                       [1, 1, 1, 1, 1]
66   

---
13                      [1, 1, 1, 1, 1]
57    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
60                   [1, 1, 1, 0, 1, 1]
11                               [1, 1]
59                   [1, 1, 1, 1, 1, 1]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 486614   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   486614   

                                             results  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
---
131             [1, 1, 1, 1, 1, 1, 1, 1]
20                       [1, 1, 0, 1, 1]
134             [1, 1, 0, 1, 1, 1, 1, 1]
22              [1, 0, 1, 1, 1, 1, 1, 1]
125    [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]
128                   [1, 1, 1, 0, 1, 1]
132                     

---
160    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
10                    [0, 0, 0, 0, 0]
174                            [1, 1]
101    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
137    [1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
175                            [0, 1]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 56215    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018    56215   

                                             results  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...  
---
13                      [1, 1, 1, 1, 1]
57    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
60                   [1, 1, 1, 0, 1, 1]
11                               [1, 1]
59                   [1, 1, 1, 1, 1, 1]
Name: results, dtype: object
---*
        

course-v1:spbu+SOCLAB+spring_2018 243054   [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   243054   

                                             results  
0  [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
---
2                       [0, 0, 0, 0, 0]
6              [0, 0, 0, 0, 0, 0, 0, 0]
41    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
49                   [1, 1, 1, 0, 1, 1]
4                                [0, 0]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 492510   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   492510   

                                             results  
0  [0, 0, 0, 0, 0, 0, 0, 0,

0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
---
196    [1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
148                   [1, 1, 1, 1, 0]
193          [1, 0, 1, 1, 1, 0, 0, 1]
152          [1, 0, 0, 0, 1, 0, 0, 1]
149                            [1, 1]
150    [0, 1, 0, 1, 1, 1, 1, 0, 0, 0]
146    [1, 1, 1, 0, 1, 1, 0, 1, 1, 1]
194                   [1, 1, 1, 1, 1]
Name: results, dtype: object
---*
                                                                                     results
course_id                         user_id                                                   
course-v1:spbu+SOCLAB+spring_2018 559517   [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, ...
                           course_id  user_id  \
0  course-v1:spbu+SOCLAB+spring_2018   559517   

                                             results  
0  [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, ...  


In [113]:
a=[0,1,2]
b=a
print(a.sum(b))

AttributeError: 'list' object has no attribute 'sum'